In [9]:
import pandas as pd
data = pd.read_csv('C:\\Users\\hp\\Desktop\\Kunal_work\\SIH\\internship.csv')

In [10]:
data.head()

,internship_title,company_name,location,start_date,duration,stipend
0,Java Development,SunbaseData,Work From Home,Immediately,6 Months,"₹ 30,000 /month"
1,Accounting and Finance,DAKSM & Co. LLP,Noida,Immediately,6 Months,"₹ 5,000-10,000 /month"
2,Sales & Digital Marketing,Bharat Natural Elements Private Limited,Bangalore,Immediately,6 Months,"₹ 5,000 /month"
3,Social Entrepreneurship,Hamari Pahchan NGO,Work From Home,Immediately,6 Months,Unpaid
4,Videography & Photography,Esquare Lifestyle,Bangalore,Immediately,6 Months,"₹ 12,000 /month"


In [11]:
data.shape

(6485, 6)

In [13]:
df = data

In [15]:
import pandas as pd
from datetime import datetime

# Load CSV
df = data

# Internship Title & Company
df['internship_title'] = df['internship_title'].str.strip()
df['company_name'] = df['company_name'].str.strip()

# Location Cleaning
df['location'] = df['location'].str.strip().str.title()

# Remote Flag
df['is_remote'] = df['location'].str.contains("work from home", case=False)
df['location'] = df['location'].replace("Work From Home", "Remote")

# Start Date Handling
def parse_start_date(val):
    if pd.isna(val):
        return None
    if "immediate" in val.lower():
        return datetime.today().date()
    try:
        return pd.to_datetime(val).date()
    except:
        return None

# df['start_date'] = df['start_date'].astype(str).apply(parse_start_date)

# Duration in Months
df['duration_months'] = df['duration'].str.extract(r'(\d+)').astype(float)

# Stipend Parsing
import re

def parse_stipend(s):
    if pd.isna(s): 
        return (0, 0)
    s = str(s).lower()
    if "unpaid" in s:
        return (0, 0)
    
    # Extract all continuous digit groups (with commas allowed)
    nums = re.findall(r'\d[\d,]*', s)
    nums = [int(x.replace(',', '')) for x in nums]

    if len(nums) == 1:
        return (nums[0], nums[0])
    elif len(nums) >= 2:
        return (nums[0], nums[1])
    else:
        return (0, 0)
df['stipend_avg'] = (df['stipend_min'] + df['stipend_max']) / 2

# Apply again
df[['stipend_min','stipend_max']] = df['stipend'].apply(lambda x: pd.Series(parse_stipend(x)))


# print(df[['internship_title','company_name','location','is_remote','start_date','duration_months','stipend_min','stipend_max']].head())


In [14]:
df.tail()

,internship_title,company_name,location,start_date,duration,stipend
6480,Project Management,Special Situation Advisors India Private Limited,Navi Mumbai,Immediately,6 Months,"₹ 7,000-10,000 /month"
6481,Dietetics/Nutrition,Special Situation Advisors India Private Limited,Navi Mumbai,Immediately,6 Months,"₹ 7,000-10,000 /month"
6482,Wealth Management,Special Situation Advisors India Private Limited,Navi Mumbai,Immediately,6 Months,"₹ 7,000-10,000 /month"
6483,Sales,Special Situation Advisors India Private Limited,Panvel,Immediately,6 Months,"₹ 7,000-10,000 /month"
6484,Campus Ambassador,Internshala,Work From Home,Not specified,6 Months,₹2000


In [5]:
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
load_dotenv()
url = os.getenv('DATABASE_URL')
if url.startswith("postgres://"):
    url = url.replace("postgres://", "postgresql+psycopg2://", 1)

engine = create_engine(url)
with engine.connect() as conn:
    print(conn.execute(text("SELECT 1")).scalar())


1


In [ ]:
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

# env se URL lena
load_dotenv()
url = os.getenv('DATABASE_URL')

# Neon URL ko psycopg2 compatible banana
if url.startswith("postgres://"):
    url = url.replace("postgres://", "postgresql+psycopg2://", 1)

# engine create
engine = create_engine(url)

# CREATE TABLE query
create_table_query = """
CREATE TABLE IF NOT EXISTS internships (
    id SERIAL PRIMARY KEY,
    internship_title TEXT,
    company_name TEXT,
    location TEXT,
    start_date DATE,
    duration TEXT,
    stipend TEXT,
    is_remote BOOLEAN,
    duration_months INT,
    stipend_min INT,
    stipend_max INT
);
"""

# query execute
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()   # SQLAlchemy me commit karna jaruri hai
    print("✅ Table 'internships' created successfully!")


✅ Table 'internships' created successfully!


In [27]:
alter_query = """
ALTER TABLE internships
ADD COLUMN stipend_avg INT;
"""

with engine.connect() as conn:
    conn.execute(text(alter_query))
    conn.commit()
    print("✅ Column stipend_avg added!")


✅ Column stipend_avg added!


In [28]:
df = df[['internship_title','company_name','location','is_remote',
         'start_date','duration_months','stipend_min','stipend_max','stipend_avg']]

In [73]:
df.head()

,internship_title,company_name,location,is_remote,start_date,duration_months,stipend_min,stipend_max,stipend_avg
0,Java Development,SunbaseData,Remote,False,2025-08-28,6.0,30000,30000,30000.0
1,Accounting and Finance,DAKSM & Co. LLP,Noida,False,2025-08-28,6.0,5000,10000,7500.0
2,Sales & Digital Marketing,Bharat Natural Elements Private Limited,Bangalore,False,2025-08-28,6.0,5000,5000,5000.0
3,Social Entrepreneurship,Hamari Pahchan NGO,Remote,False,2025-08-28,6.0,0,0,0.0
4,Videography & Photography,Esquare Lifestyle,Bangalore,False,2025-08-28,6.0,12000,12000,12000.0


In [30]:
df.to_sql("internships", engine, if_exists="append", index=False)

print("✅ Data inserted successfully!")

✅ Data inserted successfully!


In [87]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM internships LIMIT 5;"))
    for row in result:
        print(row)


(6, 'English Curriculum Writing', 'Team Everest', 'Remote', datetime.date(2025, 8, 28), False, 6, 0, 0, 0, None)
(7, 'Search Engine Optimization (SEO)', 'Global Trend', 'Remote', datetime.date(2025, 8, 28), False, 6, 5000, 5000, 5000, None)
(8, 'Digital Dreamweaver: Internship Odyssey In Graphic Design', 'Global Trend', 'Remote', datetime.date(2025, 8, 28), False, 6, 7000, 7000, 7000, None)
(9, 'Graphic Design', 'Expedify', 'Remote', datetime.date(2025, 8, 28), False, 6, 10000, 15000, 12500, None)
(10, 'Campus Ambassador', 'Internshala', 'Remote', None, False, 6, 2000, 2000, 2000, None)


In [ ]:
# "SELECT internship_title, company_name, stipend_max
# FROM internships
# ORDER BY stipend_max DESC
# LIMIT 5;
# "

In [ ]:
# df = df.drop(columns=["stipend_avg"], errors="ignore")


In [35]:
with engine.connect() as conn:
    conn.execute(text("ALTER TABLE internships DROP COLUMN duration;"))
    conn.execute(text("ALTER TABLE internships DROP COLUMN stipend;"))
    conn.commit()
    print("✅ Columns drop ho gaye!")


✅ Columns drop ho gaye!


In [19]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("sampl-Resume.pdf")
docs = loader.load()
print(docs[0].page_content)
document = docs


First Name Last Name 
 123 Maple Dr.   617-555-1234 
 Anywhere, MA 02116  email@email.com   
 
 
Profile 
This is where you write what work you’d like to do and what motivates you. This is how 
you set the tone of your resume. Write what do you most want to the person reading your 
resume to know.  
 
 
Experience 
01/2015-06/2017    Host 
 TFI Fridays – Boston, MA 
 Describe the tasks and responsibilities of the job; your 
accomplishments during your time 
 Be specific and use numbers to show growth or 
accomplishments 
 List your jobs in reverse chronological order starting with most 
recent and working your way back  
 Use bullet points, but don’t end your sentence with punctuation 
in the bullet points 
 
 
01/2013-12/2014    Food Service Management Intern 
 Big Company LLC – Boston, MA 
 Include volunteer or internships that you’ve had that might be 
relevant to the job 
 
 
 
Education 
  2015           Associates Degree: Business Administration 
Roxbury Community College- B

In [22]:
def clean_resume_text(text):
    # remove extra newlines, special chars
    text = text.replace("\n", " ").replace("\t", " ")
    return text

resume_text = clean_resume_text(document[0].page_content)


In [23]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
import os

# 1. Define your Pydantic schema
class SkillsOutput(BaseModel):
    skills: List[str]

# 2. Setup environment and model
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=GROQ_API_KEY, model_name="Gemma2-9b-It")

# 3. Create a structured output parser from Pydantic model
parser = PydanticOutputParser(pydantic_object=SkillsOutput)

# 4. Create prompt template with format instructions from parser
prompt = ChatPromptTemplate.from_template(
    """
You are an expert HR assistant. Extract ONLY the technical and professional skills
from the following resume text. Normalize them into a clean Python list.

Resume:
{resume_text}

"""
# {format_instructions}
)
# prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# prompt -> llm (with structured output binding) -> pydantic parser
chain = prompt | llm.with_structured_output(SkillsOutput)

# 6. Example resume text
# resume_text = """
# John Doe is a Data Scientist with experience in Python, SQL, Machine Learning,
# AWS, and project management. Strong communicator with leadership skills.
# """

# 7. Run chain - input is dict with variable for prompt
skills_out = chain.invoke({"resume_text": resume_text})

print("Extracted skills:", skills_out.skills)


Extracted skills: ['Food Service Management', 'Hosting']


In [4]:
alter_query = """
ALTER TABLE internships
ADD COLUMN skills TEXT[];
"""
with engine.connect() as conn:
    conn.execute(text(alter_query))
    conn.commit()
    print("✅ Internship table me 'skills' column add ho gaya!")


NameError: name 'engine' is not defined

In [17]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Extract ONLY the technical and professional skills required from the following internship description.
Output as a clean Python list.

Internship:
{internship_text}
""")

chain = prompt | llm.with_structured_output(SkillsOutput)

# Example run for 1 internship
sample_text = df['internship_title'][0] + " " + df['company_name'][0]
skills_out = chain.invoke({"internship_text": sample_text})
print("Internship skills:", skills_out.skills)


NameError: name 'llm' is not defined

In [88]:
with engine.connect() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
    conn.commit()
    print("✅ pgvector extension enabled!")


✅ pgvector extension enabled!


In [3]:
alter_vector_query = """
ALTER TABLE internships
ADD COLUMN IF NOT EXISTS embedding VECTOR(384);
"""
with engine.connect() as conn:
    conn.execute(text(alter_vector_query))
    conn.commit()
    print("✅ 'embedding' column added!")


NameError: name 'engine' is not defined

In [90]:
from sentence_transformers import SentenceTransformer

# Pretrained HuggingFace model
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


c:\Users\hp\Desktop\Kunal_work\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Agar tumne abhi tak internships.skills fill nahi kiya hai, 
# toh yaha abhi internship_title + company_name use kar lo
df['text_for_embedding'] = df['internship_title'] + " " + df['company_name']    

# Embeddings banalo
embeddings = embedder.encode(df['text_for_embedding'].tolist(), convert_to_numpy=True)
print("✅ Embeddings generated:", embeddings.shape)


✅ Embeddings generated: (6485, 384)


In [2]:
engine.connect()

NameError: name 'engine' is not defined

In [96]:
import numpy as np

with engine.begin() as conn:
    for i, row in df.head(200).iterrows():   # 👈 sirf pehle 20 rows lenge
        emb = embeddings[i].astype(float)
        emb_str = "[" + ",".join([str(x) for x in emb]) + "]"  # pgvector format

        query = text("""
            UPDATE internships
            SET embedding = :embedding
            WHERE internship_title = :title AND company_name = :company
        """)
        conn.execute(query, {
            "embedding": emb_str,
            "title": row['internship_title'],
            "company": row['company_name']
        })

print("✅ First 20 embeddings stored in DB!")


✅ First 20 embeddings stored in DB!


import numpy as np

with engine.begin() as conn:
    for i, row in df.head(20).iterrows():   # 👈 sirf pehle 20 rows lenge
        emb = embeddings[i].astype(float)
        emb_str = "[" + ",".join([str(x) for x in emb]) + "]"  # pgvector format

        query = text("""
            UPDATE internships
            SET embedding = :embedding
            WHERE internship_title = :title AND company_name = :company
        """)
        conn.execute(query, {
            "embedding": emb_str,
            "title": row['internship_title'],
            "company": row['company_name']
        })

print("✅ First 20 embeddings stored in DB!")


In [ ]:
# import numpy as np

# with engine.begin() as conn:
#     for i, row in df.iterrows():
#         emb = embeddings[i].astype(float)
#         emb_str = "[" + ",".join([str(x) for x in emb]) + "]"  # pgvector format

#         query = text("""
#             UPDATE internships
#             SET embedding = :embedding
#             WHERE internship_title = :title AND company_name = :company
#         """)
#         conn.execute(query, {
#             "embedding": emb_str,
#             "title": row['internship_title'],
#             "company": row['company_name']
#         })

# print("✅ All embeddings stored in DB!")


KeyboardInterrupt: 

In [20]:
# Candidate ke resume se nikle skills
candidate_skills = skills_out.skills
print("Candidate Skills:", candidate_skills)

# Text join karke ek string bana lo
candidate_text = " ".join(candidate_skills)

# Embedding banao
candidate_embedding = embedder.encode([candidate_text], convert_to_numpy=True)[0]

print("✅ Candidate embedding ready:", candidate_embedding.shape)


NameError: name 'skills_out' is not defined

In [6]:
import numpy as np
from sqlalchemy import text

# Embedding ko pgvector format me convert
cand_emb_str = "[" + ",".join([str(x) for x in candidate_embedding]) + "]"

# Top 5 closest internships
query = text("""
    SELECT id, internship_title, company_name, location, stipend_min, stipend_max,
           1 - (embedding <=> :cand_emb) AS similarity
    FROM internships
    WHERE embedding IS NOT NULL
    ORDER BY embedding <=> :cand_emb
    LIMIT 5;
""")

with engine.connect() as conn:
    results = conn.execute(query, {"cand_emb": cand_emb_str}).fetchall()

print("🎯 Top Internship Matches:")
for r in results:
    print(r)


NameError: name 'candidate_embedding' is not defined